In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import seaborn as sns
import numpy as np

In [3]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [19]:
# Hyper Parameters
lr = 0.01
bs = 128
num_steps=2000
display_steps = 100

# Network Parameters
n_in = 784
n_h1 = 256
n_h2 = 256
n_out = 10

In [20]:
# NO NEED TO DEFINE PLACEHOLDERS
# TENSORFLOW HIGH LEVEL API HANDLES EVERYTHING ABOUT FEEDING INPUTS 

## DEFINE AN INPUT FUNCTION FOR THE ESTIMATOR API

input_fn = tf.estimator.inputs.numpy_input_fn(x={'images':mnist.train.images}, y=mnist.train.labels,
                                             batch_size=bs, num_epochs=None, shuffle=True)

In [21]:
## define network

def mlp_high(x_dict):
    # TF Estimator input is a dict
    x = x_dict['images']
    
    # TF Layers has many layers implemented. Dense layer is the same as fully connected layer
    layer1 = tf.layers.dense(x, n_h1, activation=tf.nn.relu)
    layer2 = tf.layers.dense(layer1, n_h2, activation=tf.nn.relu)
    
    out = tf.layers.dense(layer2, n_out)
    
    return out
    

In [22]:
## Define a model_fn for the TF Estimator

def model_fn(features, labels, mode):
    
    # Call our neural network
    logits = mlp_high(features)
    
    # Predictions 
    pred_classes = tf.argmax(logits, axis=1)
    pred_probas = tf.nn.softmax(logits)
    
    # if mode == prediction, return 
    if mode==tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes)
    
    # define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, 
                                                                        labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    
    # Evalutate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
    
    ## TF Estimators requires to return a EstimatorSpec, that specify
    ## the different ops for training, evaluating
    
    estim_specs = tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=pred_classes,
        loss=loss_op,
        train_op=train_op,
        eval_metric_ops={'accuracy': acc_op})
    
    return estim_specs

In [23]:
## Build the estimator
model = tf.estimator.Estimator(model_fn)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp619j014z', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f26bd819ba8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [24]:
# Train the model
model.train(input_fn, steps=num_steps)   #### model.train means train mode

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp619j014z/model.ckpt.
INFO:tensorflow:loss = 2.331015, step = 1
INFO:tensorflow:global_step/sec: 106.186
INFO:tensorflow:loss = 1.7922153, step = 101 (0.943 sec)
INFO:tensorflow:global_step/sec: 120.143
INFO:tensorflow:loss = 1.1786933, step = 201 (0.832 sec)
INFO:tensorflow:global_step/sec: 106.928
INFO:tensorflow:loss = 0.7930119, step = 301 (0.935 sec)
INFO:tensorflow:global_step/sec: 120.214
INFO:tensorflow:loss = 0.61279184, step = 401 (0.834 sec)
INFO:tensorflow:global_step/sec: 94.3153
INFO:tensorflow:loss = 0.5714313, step = 501 (1.058 sec)
INFO:tensorflow:global_step/sec: 103.111
INFO:tensorflow:loss = 0.46729916, step = 601 (0.972 sec)
INFO:tensorflow:global_step/sec: 79.3585
INFO:tensorflow

In [25]:
## Evaluate the model
## Define the input function for evaluation

input_fn = tf.estimator.inputs.numpy_input_fn(x={'images':mnist.test.images}, y=mnist.test.labels,
                                             batch_size=bs, shuffle=False)
model.evaluate(input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-12-19-11:15:45
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp619j014z/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-12-19-11:15:45
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.917, global_step = 2000, loss = 0.29462704
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2000: /tmp/tmp619j014z/model.ckpt-2000


{'accuracy': 0.917, 'loss': 0.29462704, 'global_step': 2000}